In [ ]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict

import dvu
import matplotlib.pyplot as plt
import pandas as pd
from os.path import join
import os.path
import fitz
from tqdm import tqdm
import pathlib
import imodelsx.llm
import json
import requests
import numpy as np
import openai
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()
plt.style.use('default')
dvu.set_style()

df = pd.read_csv('../data/main_updated.csv')

### Extract text from pdfs

In [ ]:
d = df[
    (df["found_paper (0=no, 1=yes, 2=low-qual)"] == 1)
    | (df["found_paper (0=no, 1=yes, 2=low-qual)"] == 2)
]
for i, row in tqdm(d.iterrows()):
    paper_file = join("../papers", str(row.id) + ".pdf")
    if pathlib.Path(paper_file).exists():
        with fitz.open(paper_file) as doc:  # open document
            text = chr(12).join([page.get_text() for page in doc])
            pathlib.Path(join("../papers", str(row.id) + ".txt")).write_bytes(
                text.encode()
            )

### Ask questions about the text

In [ ]:
# llm = imodelsx.llm.get_llm("gpt-3.5-turbo-0613")
llm = imodelsx.llm.get_llm("gpt-4-0613")

In [ ]:
functions = [
    {
        "name": "get_paper_study_distribution_by_gender",
        "description": "Get the distribution of patients in this study accordong to gender",
        "parameters": {
            "type": "object",
            "properties": {
                "num_male": {
                    "type": "int",
                    "description": "The number of male patients in the study",
                },
                "num_female": {
                    "type": "int",
                    "description": "The number of female patients in the study",
                }
            },
            "required": ["num_male", "num_female"],
        },
    },

]
messages = [{"role": "user", "content": "What's the weather like in Boston?"}]

In [ ]:
msg = llm(messages, functions=functions, return_str=False)
print(msg.get('function_call'))